In [37]:
!pip3 install matplotlib numpy pylzma ipykernel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 51.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for pylzma: filename=pylzma-0.5.0-cp310-cp310-linux_x86_64.whl size=222322 sha256=89cad859e08419f85b5f641d62743b5f615d86f2f499c1a18aaf54228353ed0f
  Stored in directory: /root/.cache/pip/wheels/74/c9/02/91112815e838f544c1d46fda071241e454694579d022751d2b
Successfully built pylzma


******************START OF CODE******************

Bigram model and playing around ::

In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#print(device)

block_size = 8
batch_size = 4

In [29]:
with open("wizard of oz for llm.txt", 'r', encoding='utf-8') as f: #open smaple training data file as read with utf 8 encoding
  text = f.read()
chars = sorted(set(text)) #creates a sorted list of all used characters
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [30]:
string_to_int = { ch:i for i,ch in enumerate(chars) } #mappings for str to int and vice versa
int_to_string = { i:ch for i,ch in enumerate(chars) } # above
encode = lambda s: [string_to_int[c] for c in s] #encoder
decode = lambda l: ''.join([int_to_string[i] for i in l]) #decoder

#example NOT actual code, comment out when not testing/demo-ing =>

'''encoded_hello = encode('hello') #encode string 'hello' to integars
decoded_hello = decode(encoded_hello)
print(encoded_hello)
print(decoded_hello)'''

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([79, 32, 72,  5, 71,  1, 66, 67,  1, 73, 71, 57, 22,  1, 66, 67,  1, 73,
        71, 57,  1, 53, 72,  1, 53, 64, 64, 10,  1, 43, 60, 57,  1, 55, 60, 61,
        64, 56, 70, 57, 66,  1, 75, 67, 66,  5, 72,  1, 64, 57, 72,  1, 65, 57,
         1, 71, 72, 67, 68,  1, 72, 57, 64, 64, 61, 66, 59,  1, 72, 53, 64, 57,
        71,  0, 67, 58,  1, 72, 60, 57,  1, 35, 53, 66, 56,  1, 67, 58,  1, 38,
        78, 10,  1, 32,  1, 63, 66, 67, 75,  1])


In [31]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

x, y = get_batch('train')
print('inputs: ')
# print(x.shape)
print(x)
print('targets: ')
print(y)

tensor([ 85872, 135534,  32309,  67503])
inputs: 
tensor([[60, 57, 70,  1, 65, 67, 73, 72],
        [67, 75,  1, 55, 53, 70, 57, 64],
        [ 5, 56,  1, 63, 66, 67, 75, 66],
        [61, 63, 57,  1, 72, 67,  1, 59]], device='cuda:0')
targets: 
tensor([[57, 70,  1, 65, 67, 73, 72, 60],
        [75,  1, 55, 53, 70, 57, 64, 57],
        [56,  1, 63, 66, 67, 75, 66,  1],
        [63, 57,  1, 72, 67,  1, 59, 67]], device='cuda:0')


In [39]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocabulary_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocabulary_size, vocabulary_size)

  def forward(self, index, targets=None):
    logits = self.token_embedding_table(index)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    # index is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self.forward(index)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence
      index = torch.cat((index, index_next), dim=1) # (B, T+1)
    return index

model = BigramLanguageModel(vocabulary_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


7z]j?,ge﻿S(YZ:].﻿6Bq(0
hc41Aw﻿011-_J7;v"B5rI?3hn (KaU&KBqKBQHo0?,zGT m﻿0v"e-HEh1zzuApht-RltTEP?oB459C,!:mf"M8!rGhnVZlMhZIo﻿8 Nar5cld9ZqgjqfslM8-)"gYs:HF],P(J[bkuv;AI,s"J:﻿QR4R-RXy;UoRkCfYbUC2(;M8lf"U;sgCA&,NQaSXzukt-A
lXXXxdoK]cEt40uC,3F&V05RlaY[VO25)KyDyt-RlO2albAwYbB!V1pQ﻿x7f[D_dYT-0-TPTmp:﻿XUBXKQ'dTelX]cqk323vhW.
VYGg tU)?Jvr"RP
(SeP&ta-;OHFJ[q41!pzy﻿!DW.NL0vPPVDi'
F]"-K!.'GM_T-J,8r5_;VG&pj"MdV17JG?' OLKyw(T1"J7VU;3G7,[zvHL﻿r-6eJ[Qpo9﻿UUtdkEx3U;]ZRdDGM"ZZ!T9Vc3﻿y::45jf;-6Gy6,ol3 56CKKK,mJgQ-j


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

torch-examples / playing about

In [33]:
import torch

In [34]:
randint = torch.randint(-100, 100, (6,))
randint

tensor([-14,  60,   4, -99, -53, -86])

Embedding Vectors

In [35]:
import torch
import torch.nn as nn

# initialise an embedding layer
vocab_size = 1000
embedding_dim = 100
embedding = nn.Embedding(vocab_size, embedding_dim)

# create some input indices
input_indices = torch.LongTensor([1, 5, 3, 2])

# apply the embedding layer
embedded_output = embedding(input_indices)

# the output will be a tensor of shape (4, 100), where 4 is the number of inputs
# and 100 is the dimensionality of the embedding vectors
print(embedded_output.shape)

torch.Size([4, 100])


In [36]:


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print('when input is', context, 'target is', target)

when input is tensor([79]) target is tensor(32)
when input is tensor([79, 32]) target is tensor(72)
when input is tensor([79, 32, 72]) target is tensor(5)
when input is tensor([79, 32, 72,  5]) target is tensor(71)
when input is tensor([79, 32, 72,  5, 71]) target is tensor(1)
when input is tensor([79, 32, 72,  5, 71,  1]) target is tensor(66)
when input is tensor([79, 32, 72,  5, 71,  1, 66]) target is tensor(67)
when input is tensor([79, 32, 72,  5, 71,  1, 66, 67]) target is tensor(1)
